In [57]:
import re
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import contractions
from gensim.models import Word2Vec
import os


In [58]:
class Tweet(object):
    def __init__(self, filename, professionalism=0):
        self.filename = filename
        with open(filename, "r", encoding="utf-8") as text:
            self.text = text.read()
        self.professionalism = professionalism
    
    def clean(self):
        edit = self.text.lower()
        edit = re.sub("@[A-Za-z0-9_]+","", edit)
    ### edit = re.sub("#","", edit)
        edit = re.sub(r"http\S+", "", edit)
        edit = re.sub(r"www.\S+", "", edit)    
        edit = re.sub("[^a-z0-9'’#]"," ", edit)
        
        expanded_words = []
        for word in edit.split():
            expanded_words.append(contractions.fix(word))
        edit = ' '.join(expanded_words)

### splitting, joining, and splitting again is necessary because the contraction fixer
### stores both resulting words in one token.

        edit = edit.split()

        sw = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an',
              'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been',
              'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't",
              'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't",
              'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from',
              'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself',
              'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've",
              'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's",
              'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of',
              'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd",
              "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than',
              'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then',
              'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up',
              'very', 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were',
              "weren't", 'what', "what's", 'when', "when's", 'where', "where's", 'which',
              'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would',
              "wouldn't", 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours',
              'yourself', 'yourselves']

        edit = [w for w in edit if not w in sw]
        self.tokenized = edit
        
        stemmed = [ps.stem(i) for i in edit]

        return " ".join(stemmed)

In [132]:
tweets = []
df = pd.DataFrame(tweets, columns=["Text", "Cleaned"])

train_number = 1967
for i in range(len(os.listdir("tweet_text")))[:train_number]:
    tweets.append(Tweet("tweet_text/" + os.listdir("tweet_text")[i]))
    new_df = pd.DataFrame([[tweets[i].text, tweets[i].clean()]], \
                          index = [re.sub("tweet_text/", "", tweets[i].filename)], \
                          columns = ["Text", "Cleaned"])
    df = pd.concat([df, new_df], axis=0)

# I need to find out why I need to add parentheses to the tweet.preprocess part (otherwise bound error)

bag = CountVectorizer().fit_transform(df["Cleaned"])

tfidf = TfidfVectorizer().fit_transform(df["Cleaned"])

column_list = []
for i in range(bag.todense().shape[1]):
    column_list.append("Bag of Words " + str(i))
bag_dense_df = pd.DataFrame(bag.todense(), columns = column_list, index =\
                              [re.sub("tweet_text/", "", tweets[i].filename) for i in range(0, len(tweets))])

column_list = []
for i in range(tfidf.todense().shape[1]):
    column_list.append("TFIDF " + str(i))
tfidf_dense_df = pd.DataFrame(tfidf.todense(), columns = column_list, index =\
                              [re.sub("tweet_text/", "", tweets[i].filename) for i in range(0, len(tweets))])


labels = pd.read_csv("labels.csv", index_col=0)

labels["Professionalism"] = labels["Professionalism"].apply(int)

df = pd.concat([df, bag_dense_df, tfidf_dense_df], axis=1)

df = pd.concat([labels, df], axis=1)
df.to_csv("test_csv.csv")

                        Professionalism
Name                                   
silyoli14 6.txt                       1
201grip 1.txt                         0
thoughtsfreeing 38.txt                1
Grindpay365x 27.txt                   0
JambaaJuicee 40.txt                   1
...                                 ...
JambaaJuicee 38.txt                  -1
Domhausen 6.txt                      -1
Cellk8 41.txt                        -1
etheralbri 36.txt                     0
eternallife333 3.txt                  0

[500 rows x 1 columns]
